# Reinforcment Learning: An Introduction to the underlying Concepts

This blogpost is intended to give the reader a baseline understanding of the concepts underlying modern reiforcment learning algorithms. As such, this post serves as a starting point, outlining different approaches taken by various reinforcment learning algorithms, in order to enable further in depth study of the subject and of algorithms only briefly touched upon here. 

## Why bother with Reinforcment Learning?

One only needs to look at recent historic successes of reinforcment learning algorithms to see the various uses for such techniques. The most well known success is probably AlphaGo, a computer programm developed by researchers at DeepMind, which has been shown to surpass the abilitys of the best professional human Go players in 2017. Other success stories are to be found in the realm of esports. OpenAI developed bots able to beat professional teams at the highly competetive video game Dota 2 in a 5vs5 game in 2019.

These examples give us an understanding of the immense power of reinforcement learning with regards to strategic planning. It is not hard to imagine how such strengths could help advance many fields, ranging from robotics to distributed systems. Therefore, it benefits us greatly to develop a basic understanding of such learning algorithms to identify potential use cases in our daily work.

# The base Concepts

To start off, it is helpful to understand the assumptions under which reinforcment learning algorithms operate. A reinforcment learning problem is defined by both the **environment** and the **agent**, whose actions effect said environment. We can now define the interaction of both thus: given a **state** $s \in \mathcal{S}$ it is the agents task to choose an **action** $a \in \mathcal{A}$. The environment will then give a **reward** $r \in  \mathcal{R}$ to the agent and determine the next state $s' \in \mathcal{S}$ based on the state $s$ and the action $a$ chosen by the agent. Herein $\mathcal{S}$ constitutes the set of all possible states, $\mathcal{A}$ the set of all possible actions and $\mathcal{R}$ the set of all possible rewards. This relationship can also be easily illustrated.

In [ ]:
# insert picture of agent-environment duality here

If we now define $s_1$ as the starting state and $r_t, s_t, a_t$ as the state, action or reward at timestep $t = 1, 2, \ldots, T$ with the condition that $s_T$ is a terminal state, we can show the entire interaction sequence between agent and environment, also called one **episode** (sometimes called "trial" or "trajectory"), as follows:

\begin{equation}
    \tau = (s_1, a_1, r_2, s_2, a_2, r_3, \ldots s_{T-1}, a_{T-1}, r_{T}, s_{T})
\end{equation}

Note that sometimes $s_t, a_t, r_t$ are denoted by capital letters $S_t, A_t, R_t$.

## The Environment

The environment can be modeled using a transition function $P(s', r | s, a)$ which gives us the probability to go to $s'$ as the next state with reward $r$ under the condition that the current state is $s$ and the agent chose action $a$. From this functin we can also derive the state-transition function $P(s' | s, a)$ as well as the reward function $R(r | s, a)$.

\begin{align}
    P(s' | s, a) &= P_{s, s'}^a = \sum_{r \in \mathcal{R}} P(s', r | s, a) \\
    R(r | s, a) &=  \mathbb{E} [ r | s, a ] = \sum_{r \in \mathcal{R}} r \sum_{s' \in \mathcal{S}} P(s', r | s, a) \\
\end{align}

$R$ is the expected value for the reward $r$ given $s$ and $a$. A singel **transition** in the environment can thus be specified by a tuple $(s, a, r, s')$.


### Model-based vs Model-free

Depending on how much the agent knows about the **transition function** $P$ and the **reward function** $R$ we can already classify reinforcment learning algorithms as:

- Model-based: The agent has knowledge of the environment. This can be further subdivided:
    - Model is given: The agent knows $P$ and $R$ perfectly. AlphaGo can be seen as one such example as the programm is perfectly aware of the Go rules.
    - Model is learned: While $P$ and $R$ are not known at the start, the agent learns these functions during the learning process.
- Model-free: The agent does not have and does not attempt to learn $P$ and $R$. (Atari games as example?)



## The Agent

As stated above, the agents goal is to choose an action $a$ given a state $s$. We can formalize this using a function $\pi$ also known as **policy**. The policy can take one of two forms:

- deterministic: $a = \pi(s)$ (also written as $\mu (s)$)
- stochastic: $\pi(a | s)$ is the chance that $a$ is chosen under the condition $s$. In this case the derived action is denoted by $a \sim \pi(\cdot | s)$.

The goal of our reinforcment learning algorithms will always be to develop a policy $\pi$ which is as close as posible to the optimal policy $\pi^*$. One of the main differences in the various algorithms is the method used to determine this policy $\pi$.

To gain a better understanding of these methods it can be helpful to imagine how a human would go about formulating a strategy in such circumstances. This is espacially intuitive when looking at games as reinforcment learning problems where the programm effectivly replaces a human player.

### Value Function

One approach a human might use is to assign a value to each state based upon how "good" that state is for generating future rewards. The policy could then simply consist of taking the action which leads to the next state with the highest value weighted by probabilities. Although informal, this describes the concept of the **state-value function** $V(s)$ (also known as value function).
 
Yet before we can formally describe $V(s)$ we need to define the future reward or **return** $G_t$, which measures the discounted rewards after timestep $t$ in an episode:

\begin{align}
    G_t = r_{t+1} + \gamma r_{t+2} + \gamma^2 r_{t+3} + \ldots = \sum_{k = 0}^\infty \gamma^k r_{t+1+k}
\end{align}

The form of $G_t$ above is called the **infinite horizon discounted-return**. If we instead sumed only to a constant $T$ instead of $\infty$ we would get the **finite horizon discounted-return**. 


The factor $\gamma \in [0, 1]$ is a hyperparameter discounting rewards baesed on how far in the future they occur. Doing this has multiple practical reasons as we may prefare immeadiate benefits over drawn out ones (such as winning a game quickly). Further, this also provides mathematical advantages when encountering loops in the transition sequence and futher allows:

\begin{align}
    G_t &= r_{t+1} + \gamma r_{t+2} + \gamma^2 r_{t+3} + \ldots \\
    &= r_{t+1} + \gamma (r_{t+2} + \gamma r_{t+3} + \ldots) \\
    &= r_{t+1} + \gamma G_{t+1}
\end{align}

Using the return we can now formulate an equation for $V(s)$:

\begin{align}
    V(s) &= \mathbb{E}[G_t | s_t = s] \\
    V_\pi(s) &= \sum_{a} \pi(a | s) \sum_{s', r} P(s', r | s, a) (r + \gamma V_\pi(s'))
\end{align}

Note the dependency of $V(s)$ on an episode or a policy. The second equation here is also called a **Bellman equation**. Therefore a policy is required to determine the state-value function and we can define $V^*(s)$ as the function generated by the opotimal policy $\pi^*$. Once a function $V(s)$ is given we can derive a greedy policy $\pi$ accordingly:

\begin{align}
    \pi(s) = \text{arg} \max_{a} \sum_{s', r} P(s', r | s, a) (r + \gamma V(s'))
\end{align}


If we can find an algorithm in which $V^\pi(s)$, generated by the current policy $\pi$, converges on $V^*(s)$ we can thus derive the optimal policy $\pi^*$. This approach is realized in the **policy-iteration** algorithm which alternates between generating a state-value function from a policy and generating a policy from the state-value function.


### Action-Value Function

Instead of only evaluating states, it is also possible to evaluate state-actions pairs, resulting in the **action-value function** $Q(s, a)$. We can define $Q(s, a)$ similarly to $V(s)$ and logicaly connect both functions for a given policy:

\begin{align}
    Q(s, a) &= \mathbb{E}[G_t | s_t = s, a_t = a] \\
    V_\pi(s) &= \sum_{a} \pi(a | s) Q_\pi(s, a) \\
    Q_\pi(s, a) &= \sum_{s', r} P(s', r | s, a) (r + \gamma V_\pi(s'))
\end{align}

We can also formulate a **Bellman equation**:

\begin{align}
    Q_\pi(s, a) &= \sum_{s', r} P(s', r | s, a) (r + \gamma \sum_{a'} \pi(a' | s) Q_\pi(s', a')) \\
\end{align}

The action-value function can also be used to greedily generate a policy:

\begin{align}
    \pi(s) = \text{arg} \max_{a} Q(s, a)
\end{align}


## Approaches to Reinforcment Learning

In Theory, every problem we describe using this framework could be seen as a simple planning problem, only requiering the maximum solution to a Bellman equation and we could derive an optimal policy. In practice, however, the number of possible states and actions as well as the limited knowledge of the environment makes this unfeasible, forcing us to find other ways of converging on the optimal policy. Algorithms therefore usually optimize the state-value function, the action-value function or the policy directly. If the policy is directly optimized, it tends to consist of another function with a given set of parameters that can be optimized.
 
Since we have defined the most important terms, we can now delve deeper into specific algorithms and approaches. This will further our understanding of reinforcment learning and detail the implementation of 

In [ ]:
# maybe a list of algorithms to go to next

In [ ]:
# maybe a note on continuous state and action spaces?